In [10]:
import requests
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [11]:
import numpy as np
import pandas as pd
import json
import re
import os, sys

In [12]:
main_dir = os.path.normpath(os.getcwd() + os.sep + os.pardir)

# Prize Picks Projections Scrape

In [13]:
f = open(main_dir + "/PrizePicks/projections.json")
data = json.load(f)
pp = pd.json_normalize(data['included'])
pp1 = pp[pp['type'] == 'new_player']
pp2 = pd.json_normalize(data['data'])

pp1 = pp1.rename(columns = {'id' : 'playerid'})
pp2 = pp2.rename(columns = {'relationships.new_player.data.id' : 'playerid'})

mergedDF = pp2.merge(pp1, on = 'playerid', how = 'left')

fdf = mergedDF[['attributes.line_score','attributes.stat_type','attributes.name','attributes.league']]


In [14]:
fdf.rename(columns={'attributes.line_score':'pp_line', 'attributes.stat_type':'prop_type', 'attributes.name':'player_name', 
                   'attributes.league':'league'}, inplace=True)

In [15]:
fdf['player_name'] = fdf['player_name'].str.lower()
fdf['league'] = fdf['league'].str.lower()
fdf['prop_type'] = fdf['prop_type'].str.lower()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy

In [16]:
# changing prop types to match pinnacle
# keep adding to this until complete

# MLB PROPS
fdf['prop_type'] = np.where(fdf['prop_type']=='earned runs allowed', 'earned runs',
                           np.where(fdf['prop_type']=='strikeouts', 'total strikeouts',
                                   fdf['prop_type']))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


# Pinnacle Sports MLB WebScrape Attempt

In [17]:
# Opening JSON file
f = open(main_dir + "/Pinnacle/MLB/mlb_matchups.json")  

# returns JSON object as 
# a dictionary
data = json.load(f)

# Iterating through the json
# list
player_props = []
#test = []
for i, d in enumerate(data['data']):
    try:
        if d['special']['category'].lower() == 'player props':
            data = {}
            data['prop_name'] = d['special']['description']
            data['prop_id'] = d['id']
            data['over_id'] = d['participants'][0]['id']
            data['under_id'] = d['participants'][1]['id']
            player_props.append(data)        
                
            
    except:
        continue

# Closing file
f.close()

In [18]:
# Opening JSON file
f = open(main_dir + "/Pinnacle/MLB/mlb_straight.json")
  
# returns JSON object as 
# a dictionary
data = json.load(f)
prop_prices = []
for d in data['data']:
    try:

        row = {}
        row['prop_id'] = d['matchupId']
    
        row['over_id'] = d['prices'][0]['participantId']
        row['over_points'] = d['prices'][0]['points']
        row['over_price'] = d['prices'][0]['price']
    
        row['under_id'] = d['prices'][1]['participantId']
        row['under_points'] = d['prices'][1]['points']
        row['under_price'] = d['prices'][1]['price']   
 
        prop_prices.append(row)
    
    except:
        continue


    
# Closing file
f.close()

In [19]:
propd = pd.DataFrame(player_props)

In [20]:
prices = pd.DataFrame(prop_prices)

In [21]:
df = pd.merge(propd, prices, how='left', on=['prop_id', 'over_id', 'under_id'])

In [22]:
df.dropna(inplace=True)

In [23]:
df['min_price'] = np.where(df['over_price']<=df['under_price'], df['over_price'], df['under_price'])

In [24]:
def prop_type(string):
    type_ = re.findall('(.+?)\(', string)[1].lower().strip(')')
    return type_

def player_name(string):
    name = re.findall('(.+?)\(', string)[0].lower().strip()
    return name

In [25]:
df['player_name'] = df['prop_name'].apply(player_name)
df['prop_type'] = df['prop_name'].apply(prop_type)

In [26]:
df.drop(columns='prop_name', inplace=True)

In [27]:
df.drop_duplicates(inplace=True)

In [28]:
propd = pd.DataFrame(player_props)
prices = pd.DataFrame(prop_prices)

df = pd.merge(propd, prices, how='left', on=['prop_id', 'over_id', 'under_id'])
df.dropna(inplace=True)

df['min_price'] = np.where(df['over_price']<=df['under_price'], df['over_price'], df['under_price'])

df['player_name'] = df['prop_name'].apply(player_name)
df['prop_type'] = df['prop_name'].apply(prop_type)

df.drop(columns='prop_name', inplace=True)
df.drop_duplicates(inplace=True)

In [29]:
def get_picks(pp, pin, league):
    df = pp[pp['league']==league]
    df = df.merge(pin, how='left', on=['player_name', 'prop_type'])
    df.dropna(inplace=True)
    df.drop(columns=['prop_id', 'over_id', 'under_id', 'under_points'], inplace=True)
    df.rename(columns={'over_points':'pinnacle_line'}, inplace=True)
    
    df['favored'] = np.where(df['min_price']==df['over_price'], 'over', 'under')
    
    df['edge'] = np.where((df['pinnacle_line']<df['pp_line'])&(df['favored']=='under'), 'edge',
                         np.where((df['pinnacle_line']>df['pp_line'])&(df['favored']=='over'), 'edge',
                                 np.where(df['pinnacle_line']==df['pp_line'], 'equal',
                                         np.where((df['pinnacle_line']<df['pp_line'])&(df['favored']=='over'), 'buffer',
                                                 np.where((df['pinnacle_line']>df['pp_line'])&(df['favored']=='under'), 'buffer', 'else')))))
    df.sort_values(by='min_price', inplace=True)
    return df

In [30]:
mlb = get_picks(fdf, df, 'mlb')

In [31]:
mlb.sort_values(by='min_price', ascending=True)

,pp_line,prop_type,player_name,league,pinnacle_line,over_price,under_price,min_price,favored,edge
10,4.0,hits allowed,blake snell,mlb,4.5,135.0,-185.0,-185.0,under,buffer
29,2.0,earned runs,justin verlander,mlb,1.5,-183.0,134.0,-183.0,over,buffer
25,4.0,total strikeouts,jameson taillon,mlb,3.5,-164.0,122.0,-164.0,over,buffer
31,6.0,total strikeouts,justin verlander,mlb,5.5,-160.0,119.0,-160.0,over,buffer
52,0.5,total bases,jose trevino,mlb,0.5,-157.0,117.0,-157.0,over,equal
47,0.5,total bases,trey mancini,mlb,0.5,-144.0,108.0,-144.0,over,equal
22,0.5,total bases,bryson stott,mlb,0.5,-143.0,107.0,-143.0,over,equal
21,0.5,total bases,ha-seong kim,mlb,0.5,-142.0,107.0,-142.0,over,equal
50,0.5,total bases,josh donaldson,mlb,0.5,-140.0,105.0,-140.0,over,equal
8,6.0,total strikeouts,blake snell,mlb,6.5,104.0,-139.0,-139.0,under,buffer
